# 1. 라이브러리 불러오기

In [35]:
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import time
import re
import csv 
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

from bs4 import BeautifulSoup
from tqdm import tqdm

# 2. 수집한 리뷰들 불러오기

## 2-(1) 방문자 리뷰 불러오기

In [36]:
#추출한 방문자리뷰 csv파일 불러오기
visitor_review_list=pd.read_csv('place_visitor_review.csv',encoding='utf-8')
visitor_review=visitor_review_list[['num','name','blog or review','visitor review content']]

In [37]:
#불러온 방문자 리뷰 중 결측값 확인
visitor_review.isnull().any()

num                       False
name                      False
blog or review            False
visitor review content    False
dtype: bool

In [38]:
#불러온 방문자 리뷰 중 결측값이 있는 행 제거
visitor_review=visitor_review.dropna(axis=0)

In [39]:
#방문자 리뷰 결측값 제거 완료
visitor_review.isnull().any()

num                       False
name                      False
blog or review            False
visitor review content    False
dtype: bool

In [40]:
visitor_review

,num,name,blog or review,visitor review content
0,1,배곧 우정횟집,방문자 리뷰,"가족 모임으로 다녀왔습니다. 좋은 리뷰 평가가 많아서 일단 믿고 왔는데, 부부 사장..."
1,2,배곧 우정횟집,방문자 리뷰,주말에 어디갈까 하다가 티맵에 오이도맛집 치니 똭! 하고 우정회집이 나오더라구요. ...
2,3,배곧 우정횟집,방문자 리뷰,결혼기념일 남편이랑 다녀왔습니다.\r\n5월 제철 - 도다리 회. 밑반찬에 시키다시...
3,4,배곧 우정횟집,방문자 리뷰,거래처 사장님 추천으로 왔다가 완전 맛있게 잘 먹었어요. 음식 진짜 깔끔하고 맛있어...
4,5,배곧 우정횟집,방문자 리뷰,오이도에서 제일 멋진 횟집입니다.\r\n맛있어요.
...,...,...,...,...
3226,6,배곧 그레몽디저트샵,방문자 리뷰,출산한 아내가 가장 먹고싶다고 말한 그레몽 에그타르트!! 맛이 정말 예술입니다 ㅎ...
3227,7,배곧 그레몽디저트샵,방문자 리뷰,진짜 거짓말 안하고 동네에 친구들 놀러오면 꼭 데려가는 곳입니다 왜냐구요? 실패가 ...
3228,8,배곧 그레몽디저트샵,방문자 리뷰,인스타 갬성 카페예요~디저트가 너무 맛있고 인테리어를 엄청 신경쓰신게 보였어요🌸🌸\...
3229,9,배곧 그레몽디저트샵,방문자 리뷰,빵 디저트 엄청 좋아해서 배곧 안 가본곳이 없어요!요기 휘낭시에 겉은 진짜 바삭하고...


# 3. 전처리(특수문자, 띄어쓰기, 불용어 제거 등)

## 3-(1) 전처리 라이브러리 호출

In [41]:
#전처리를 하기위해 필요한 라이브러리 호출
import os
import sys
import konlpy
import kss
from pykospacing import Spacing
from tqdm import tqdm
import time
import pandas as pd
import re

spacing = Spacing()

## 3-(2) 기본 전처리

In [42]:
corpus = []
append = corpus.append

def normalization(cr_review):
    for i in tqdm(range(0, len(cr_review)), desc='전처리 진행율'):
        review = re.sub('\n','', str(cr_review[i]))
        # 기본 전처리
        review = re.sub('\u200b','',review) # 폭 없는 공백 제거
        review = re.sub('\xa0','',review) # Latin1 (ISO 8859-1)비 공백 공간 제거
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\;\!\-\,\_\~\$\'\"\:]', '',review) # 여러 기호 제거
        review = re.sub(r'[^가-힣-\s0-9]', '', review) # 한글과 숫자만 살리고 제거
        review = re.sub(r'\s+', ' ', review) # 다중 공백 제거
        review = re.sub(r'^\s+', '', review) # 시작 공백 제거
        # 표현 및 문법 전처리
        review = re.sub('라구요|라구여|라구용|라고염|라고여|라고용', '라고요', review)
        review = re.sub('어용|어염|어여', '어요', review)
        review = re.sub('봐용|봐염|봐여', '봐요', review)
        review = re.sub('게용|게염|게여', '게요', review)
        review = re.sub('했당', '했다', review)
        review = re.sub('았당', '았다', review)
        review = re.sub('에용|에염|에여|예염', '에요', review)
        review = re.sub('세용|세염|세여', '세요', review)
        review = re.sub('께용|께염|께여|께유', '께요', review)
        review = re.sub('해용|해염|해여|해유', '해요', review)
        review = re.sub('네용|네염|네여|네유', '네요', review)
        review = re.sub('아용|아염|아여', '아요', review)
        review = re.sub('니당', '니다', review)
        review = re.sub('괜춘', '괜찮네요', review)
        review = re.sub('이뻐', '예뻐', review)
        review = re.sub('이쁘', '예쁘', review)
        review = re.sub('이쁜', '예쁜', review)
        review = re.sub('고기집', '고깃집', review)
        review = re.sub('같아용|같아염|같아여', '같아요', review)
        review = re.sub('같네용|같네염|같네여', '같네요', review)
        review = re.sub('이구용', '이구요', review)
        review = re.sub('었 따', '었다', review)
        # 띄어쓰기
        review = spacing(review)
        # 띄어쓰기 전처리
        review = re.sub('니\s다', '니다', review)
        review = re.sub('라\s고요|라고\s요', '라고요', review)
        review = re.sub('배\s곧', '배곧', review)
        review = re.sub('또\s잇', '또잇', review)
        review = re.sub('와\s규', '와규', review)
        review = re.sub('에\s비야', '에비야', review)
        #문장 분절
        review = kss.split_sentences(review)
        append(review)
        
        
    return corpus

In [43]:
review_content = visitor_review['visitor review content']
corpus = normalization(review_content)

전처리 진행율: 100%|██████████| 3231/3231 [14:27<00:00,  3.73it/s]


## 3-(3) 네이버 플레이스 방문자 리뷰 전처리

In [44]:
#corpus와 visitor_review2의 데이터 갯수 차로 인한 오류 발생
def convert_place_review(df, corpus):
    df.dropna(inplace=True) # 데이터프레임 결측값 제거
    df.reset_index(inplace=True) # 데이터 프레임 인덱스 초기화

    for i in range(len(corpus)):
        for j in range(len(corpus[i])):
            corpus[i][j] = corpus[i][j]+'.'
    df.insert(4, 'visitor review',  corpus) # 4번째 행에 전처리된 플레이스 리뷰 삽입
    df.dropna(inplace=True) # 정규화를 하면 플레이스 리뷰의 경우 이모티콘만 있는 경우에 결측값이 되기 때문에 한번 더 결측값 제거
    df.drop(['visitor review content', 'index'], axis=1, inplace=True) # 기존 리뷰 행 삭제
    df.to_csv('processed_place_visitor_review.csv', encoding='utf-8-sig')

In [45]:
convert_place_review(visitor_review, corpus)

In [46]:
dff = pd.read_csv('processed_place_visitor_review.csv', encoding='utf-8-sig', index_col=0)
review = dff['visitor review']

# 문자열 형태로 바꿔서 풀고, [],'', , 등은 정규식으로 제거
cop = []
for i in range(len(review)):
    sr = ''
    sr += review[i]
    sr = re.sub(r'[^가-힣-\s0-9\.]', '', sr)
    cop.append(sr)

In [47]:
dff.drop(['visitor review'], axis=1, inplace=True)
dff.insert(3, 'visitor review', cop)
dff.to_csv('processed_place_visitor_review.csv', encoding='utf-8-sig')
pd.read_csv('processed_place_visitor_review.csv', encoding='utf-8-sig', index_col=0)

,num,name,blog or review,visitor review
0,1,배곧 우정횟집,방문자 리뷰,가족 모임으로 다녀 왔습니다. 좋은 리뷰 평가가 많아서 일단 믿고 왔는데 부부 사장...
1,2,배곧 우정횟집,방문자 리뷰,주말에 어디갈까 하다가 티맵에 오이도 맛집 치니 똭 하고 우정회집이 나오더라고요 역...
2,3,배곧 우정횟집,방문자 리뷰,결혼기념일 남편이랑 다녀왔습니다. 5월 제철 도다리 회 밑반찬에 시키다 시 매운탕까...
3,4,배곧 우정횟집,방문자 리뷰,거래처 사장님 추천으로 왔다가 완전 맛있게 잘 먹었어요. 음식 진짜 깔끔하고 맛있어...
4,5,배곧 우정횟집,방문자 리뷰,오이도에서 제일 멋진 횟집입니다. 맛있어요.
...,...,...,...,...
3226,6,배곧 그레몽디저트샵,방문자 리뷰,출산한 아내가 가장 먹고 싶다고 말 한 그레몽 에 그 타르트 맛이 정말 예술입니다....
3227,7,배곧 그레몽디저트샵,방문자 리뷰,진짜 거짓말 안 하고 동네에 친구들 놀러오면 꼭 데려가는 곳입니다. 왜냐구요. 실패...
3228,8,배곧 그레몽디저트샵,방문자 리뷰,인스타 갬성 카페예요. 디저트가 너무 맛있고 인테리어를 엄청 신경 쓰신 게 보였어요...
3229,9,배곧 그레몽디저트샵,방문자 리뷰,빵 디저트 엄청 좋아해서 배곧 안 가본 곳이 없어요. 요기 휘낭시에 겉은 진짜 바삭...
